# non_centered_eight example

In [1]:
import pathlib
import arviz as az
import pymc as pm
import numpy as np

In [2]:
draws = 500
chains = 4

J = 8
scores = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])
schools = np.array(
    [
        "Choate",
        "Deerfield",
        "Phillips Andover",
        "Phillips Exeter",
        "Hotchkiss",
        "Lawrenceville",
        "St. Paul's",
        "Mt. Hermon",
    ]
)

In [4]:
with pm.Model(coords={
    "school": schools,
}) as non_centered_eight:
    mu = pm.Normal("mu", mu=0, sigma=5)
    tau = pm.HalfCauchy("tau", beta=5)
    theta_tilde = pm.Normal("theta_t", mu=0, sigma=1, shape=J, dims="school")
    theta = pm.Normal("theta", mu=mu, sigma=tau, shape=J)
    y_obs = pm.ConstantData("scores", scores, dims="school")
    obs = pm.Normal("obs", mu=theta, sigma=sigma, observed=y_obs)

    idata = pm.sample_prior_predictive()
    idata.extend(pm.sample(draws, chains=chains))
    idata.extend(pm.sample_posterior_predictive(idata))
idata

Sampling: [mu, obs, tau, theta, theta_t]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 2 jobs)
NUTS: [mu, tau, theta_t, theta]


Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 94 seconds.
Sampling: [obs]


Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [5]:
# Storing the model to .nc format
idata.to_netcdf(pathlib.Path("..", "..", "data", "non_centered_eight.nc"))

WindowsPath('../../data/non_centered_eight.nc')